In [1]:
import os
import cv2
import numpy as np
import pandas as pd

import image_processor
import model_builder

from sklearn.model_selection import StratifiedKFold

In [2]:
image_size = (32, 32)
input_size = (*image_size, 1)

In [3]:
df_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
            'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
            'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
            'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
            'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [4]:
df_train = image_processor.process_image_data("data/Train", image_size, targets=df_names)

In [5]:
X = df_train.drop(columns=['target'])
y = df_train['target']

In [7]:
from tensorflow.python.keras.callbacks import EarlyStopping

X_data = X.values.reshape(-1, *image_size, 1).astype('float32')
y_data = y.factorize()[0]

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
fold = 1
cv_scores = []

for train_index, test_index in skf.split(X_data, y_data):
    print(f"\nTraining Fold {fold}...")

    X_train, X_test = X_data[train_index], X_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

    model = model_builder.build_conv_pool_model(input_size, len(df_names))
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = model.fit(X_train, y_train,
                        epochs=10,
                        batch_size=32,
                        callbacks=[early_stopping],
                        validation_data=(X_test, y_test),
                        verbose=0)

    for epoch, (val_loss, val_acc) in enumerate(zip(history.history['val_loss'], history.history['val_accuracy']), start=1):
        print(f"  Epoch {epoch}: val_loss={val_loss:.4f}, val_accuracy={val_acc:.4f}")

    score = model.evaluate(X_test, y_test, verbose=0)
    print(f"Fold {fold} Final Accuracy: {score[1] * 100:.2f}%")
    cv_scores.append(score[1])
    fold += 1


Training Fold 1...
  Epoch 1: val_loss=0.5288, val_accuracy=0.8373
  Epoch 2: val_loss=0.4473, val_accuracy=0.8490
  Epoch 3: val_loss=0.3607, val_accuracy=0.8719
  Epoch 4: val_loss=0.3333, val_accuracy=0.8800
  Epoch 5: val_loss=0.3114, val_accuracy=0.8902
  Epoch 6: val_loss=0.3091, val_accuracy=0.8904
  Epoch 7: val_loss=0.3106, val_accuracy=0.8891
  Epoch 8: val_loss=0.2956, val_accuracy=0.8934
  Epoch 9: val_loss=0.2648, val_accuracy=0.9022
  Epoch 10: val_loss=0.2731, val_accuracy=0.9049
Fold 1 Final Accuracy: 90.49%

Training Fold 2...
  Epoch 1: val_loss=0.4698, val_accuracy=0.8466
  Epoch 2: val_loss=0.4211, val_accuracy=0.8597
  Epoch 3: val_loss=0.4241, val_accuracy=0.8573
  Epoch 4: val_loss=0.3710, val_accuracy=0.8725
  Epoch 5: val_loss=0.3230, val_accuracy=0.8827
  Epoch 6: val_loss=0.3105, val_accuracy=0.8879
  Epoch 7: val_loss=0.3006, val_accuracy=0.8870
  Epoch 8: val_loss=0.2779, val_accuracy=0.9002
  Epoch 9: val_loss=0.2613, val_accuracy=0.9009
  Epoch 10: val_l

In [8]:
print("\nCross-Validation Results:")
print(f"Mean Accuracy: {np.mean(cv_scores) * 100:.2f}%")
print(f"Standard Deviation: {np.std(cv_scores) * 100:.2f}%")


Cross-Validation Results:
Mean Accuracy: 90.16%
Standard Deviation: 0.43%


In [7]:
test_df = df_test = image_processor.process_image_data("data/TestData", image_size)

In [8]:
X_test = test_df.iloc[:, 1:]
X_test_prepared = X_test.values.reshape(-1, *image_size, 1).astype('float32')

pred = model.predict(X_test_prepared)
predicted_classes = pred.argmax(axis=1)
predicted_labels = [df_names[i] for i in predicted_classes]

test_df['pred'] = predicted_labels

222/222 [==============================] - 1s 4ms/step


In [9]:
model.save("models/model_9016")

INFO:tensorflow:Assets written to: models/model_90:16/assets


INFO:tensorflow:Assets written to: models/model_90:16/assets


In [11]:
filtered_df = test_df[test_df['pred'] == '9']
filtered_df[['label', 'pred']]

,label,pred
33,Test4779.png,9
82,Test5117.png,9
123,Test5739.png,9
159,Test6581.png,9
173,Test4157.png,9
...,...,...
6829,Test5906.png,9
6835,Test5523.png,9
6869,Test5286.png,9
6973,Test5052.png,9
